In [ ]:
import os
import shutil

# Paths
dataset_path = "Dataset/crop_part1"
output_path = "organized_dataset"

# Create output directories
below_18_path = os.path.join(output_path, "below_18")
above_18_path = os.path.join(output_path, "above_18")
os.makedirs(below_18_path, exist_ok=True)
os.makedirs(above_18_path, exist_ok=True)

# Organize images
for file_name in os.listdir(dataset_path):
    if file_name.endswith(".jpg"):
        age = int(file_name.split("_")[0])  # Extract age from filename
        src_path = os.path.join(dataset_path, file_name)
        
        if age < 18:
            dest_path = os.path.join(below_18_path, file_name)
        else:
            dest_path = os.path.join(above_18_path, file_name)
        
        shutil.copy(src_path, dest_path)

print("Dataset organized into 'below_18' and 'above_18' folders!")

In [ ]:
import os
import cv2
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.applications import MobileNetV2
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import classification_report

dataset_path = "organized_dataset"

#  Define ImageDataGenerator for dynamic image loading
train_datagen = ImageDataGenerator(
    rescale=1./255,  # Normalize images
    rotation_range=20,
    width_shift_range=0.2,
    height_shift_range=0.2,
    horizontal_flip=True,
    validation_split=0.2  # 80% training, 20% validation
)

#  Create training and validation generators
train_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=16,  # Reduced batch size to avoid crashes
    class_mode='binary',
    subset='training'
)

val_generator = train_datagen.flow_from_directory(
    dataset_path,
    target_size=(224, 224),
    batch_size=16,
    class_mode='binary',
    subset='validation'
)

#  Load Pretrained MobileNetV2 Model 
base_model = MobileNetV2(weights='imagenet', include_top=False, input_shape=(224, 224, 3))
base_model.trainable = False  # Freeze base model layers

#  Add custom classification layers
x = base_model.output
x = GlobalAveragePooling2D()(x)
x = Dense(128, activation='relu')(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)  # Binary classification

model = Model(inputs=base_model.input, outputs=output)

#  Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

#  Train the model
history = model.fit(
    train_generator,
    epochs=10,
    validation_data=val_generator
)

#  Fine-tune the base model for better accuracy (optional)
base_model.trainable = True  # Unfreeze base model for fine-tuning
model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=1e-5),
              loss='binary_crossentropy', metrics=['accuracy'])

history_fine = model.fit(
    train_generator,
    epochs=5,  # Fine-tune for additional 5 epochs
    validation_data=val_generator
)

#  Save the trained model
model.save("age_classification_model.h5")
print("Model saved successfully!")


Found 7824 images belonging to 2 classes.
Found 1956 images belonging to 2 classes.
9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


/usr/local/lib/python3.11/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 135s 253ms/step - accuracy: 0.8512 - loss: 0.3541 - val_accuracy: 0.6820 - val_loss: 0.7227
Epoch 2/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 113s 231ms/step - accuracy: 0.8997 - loss: 0.2393 - val_accuracy: 0.6800 - val_loss: 0.7436
Epoch 3/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 131s 269ms/step - accuracy: 0.9084 - loss: 0.2288 - val_accuracy: 0.6984 - val_loss: 0.7454
Epoch 4/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 110s 225ms/step - accuracy: 0.9169 - loss: 0.2042 - val_accuracy: 0.7004 - val_loss: 0.6443
Epoch 5/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 112s 230ms/step - accuracy: 0.9192 - loss: 0.2157 - val_accuracy: 0.6958 - val_loss: 0.6623
Epoch 6/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 130s 265ms/step - accuracy: 0.9159 - loss: 0.2114 - val_accuracy: 0.6927 - val_loss: 0.6670
Epoch 7/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 143s 268ms/step - accuracy: 0.9222 - loss: 0.1949 - val_accuracy: 0.6871 - val_loss: 0.6637
Epoch 8/10
489/489 ━━━━━━━━━━━━━━━━━━━━ 112s 228ms/step - accuracy: 0.9245 -

Model saved successfully!


In [3]:
import os
import cv2
import numpy as np
from tensorflow.keras.models import load_model

#  Load the trained age detection model
model = load_model("age_classification_model.h5")

#  Function to predict age category
def predict_age(image_path, model):
    img = cv2.imread(image_path)
    img = cv2.resize(img, (224, 224))  # Resize to match model input
    img = img / 255.0  # Normalize
    img = np.expand_dims(img, axis=0)  # Add batch dimension

    prediction = model.predict(img)[0][0]
    return "Above 18" if prediction > 0.5 else "Below 18"

#  Function to restrict content
def restrict_content(age_category):
    if age_category == "Below 18":
        print("⚠ Access Restricted: Some content is blocked for users under 18.")
        # Example: Block access to specific websites
        blocked_websites = ["example18plus.com", "adultsite.com"]
        print("Blocked websites:", blocked_websites)

        # Example: Restrict access to specific files or videos
        restricted_files = ["restricted_movie.mp4", "explicit_content.pdf"]
        print("Restricted files:", restricted_files)
    else:
        print("Full Access: No restrictions.")

#  Test with a new image
test_image_path = "example.webp" # test image
age_category = predict_age(test_image_path, model)
print(f"Predicted Age Category: {age_category}")

# Apply content restriction
restrict_content(age_category)


1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 812ms/step
Predicted Age Category: Below 18
⚠ Access Restricted: Some content is blocked for users under 18.
Blocked websites: ['example18plus.com', 'adultsite.com']
Restricted files: ['restricted_movie.mp4', 'explicit_content.pdf']
